# Flask Application For Chatbot 

# Flask application creation

The code app = Flask(__name__) creates a Flask application instance named app.

Here's a brief explanation of what this line does:

Flask: Flask is a class from the Flask framework that represents a Flask application. By creating an instance of this class, you can create and configure your Flask application.

__name__: __name__ is a special Python variable that represents the name of the current module. When __name__ is used as the argument for the Flask class, it tells Flask to use the current module as the starting point for the application.

By assigning Flask(__name__) to the variable app, you create a Flask application object that represents your web application. This app object can then be used to define routes, handle HTTP requests, and perform other tasks required to build a web application using Flask.

# Load the trained model

The code model = keras.models.load_model('cb_model') loads a pre-trained model using the Keras load_model function from TensorFlow.

Here's a breakdown of the code:

**with open('tokenizer.pickle', 'rb') as handle::** This line opens the file 'tokenizer.pickle' in read-binary mode ('rb'). The file contains the serialized tokenizer object.

**tokenizer = pickle.load(handle):** This line uses the pickle.load() function to deserialize and load the tokenizer object from the opened file. The loaded tokenizer object is then assigned to the variable tokenizer.

**with open('label_encoder.pickle', 'rb') as enc::** This line opens the file 'label_encoder.pickle' in read-binary mode ('rb'). The file contains the serialized label encoder object.

**lbl_encoder = pickle.load(enc):** This line uses pickle.load() to deserialize and load the label encoder object from the file. The loaded label encoder object is assigned to the variable lbl_encoder.

# Running the model 

The @app.route('/') decorator associates the URL route '/' (the root URL) with the home() function. When a user visits the root URL, the home() function is executed.

The home() function returns the result of rendering the HTML template 'index.html' using the render_template() function provided by Flask. It passes an empty list chats=[] as a parameter to the template. This allows you to pass data from your Python code to the HTML template for rendering.

The @app.route('/chat', methods=['POST']) decorator associates the URL route '/chat' with the chat() function. It specifies that the function should handle POST requests.

The chat() function performs the following steps:

1. Retrieves the value of the 'message' field from the submitted form data.
2. Converts the user's text to a sequence of integers using tokenization.
3. Pads the sequence to a fixed length.
4. Uses the pre-trained model to predict the class probabilities for the padded sequence.
5. Determines the predicted tag by finding the index of the highest probability.
6. Loads intents data from the 'intents.json' file.
7. Selects a random response associated with the predicted tag.
8. Constructs a list of chat messages including the user's input and the bot's response.
9. Renders the 'index.html' template, passing the chat messages as a parameter.
The if __name__ == '__main__': block checks if the script is being executed as the main module, and if so, it runs the Flask application using app.run(), making the application accessible via a web server.

Overall, this code defines a route handler for the

In [ ]:
from flask import Flask, render_template, request
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import pickle


app = Flask(__name__)

# Configuration variables
MODEL_FILE = 'cb_model_Alphaa'
TOKENIZER_FILE = 'tokenizer.pickle'
LABEL_ENCODER_FILE = 'label_encoder.pickle'
DATA_FILE = 'Queries.json'
MAX_SEQUENCE_LENGTH = 20

# Load the trained model, tokenizer, and label encoder once when the app starts
model = keras.models.load_model(MODEL_FILE)
tokenizer = pickle.load(open(TOKENIZER_FILE, 'rb'))
lbl_encoder = pickle.load(open(LABEL_ENCODER_FILE, 'rb'))

# Function to perform speech recognition
def recognize_speech():
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    with microphone as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        print("Recognizing...")
        text = recognizer.recognize_google(audio)
        print("You said:", text)
        return text
    except sr.UnknownValueError:
        print("Could not understand audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

@app.route('/')
def home():
    return render_template('index.html', chats=[])

from flask import Flask, render_template, request, jsonify

# ...

@app.route('/chat', methods=['POST'])
def chat():
    if 'voice' in request.json:
        prompt = recognize_speech()
    else:
        prompt = request.json['message']

    if prompt is None:
        return jsonify({'message': 'Invalid input'})

    input_sequence = tokenizer.texts_to_sequences([prompt])
    padded_sequence = keras.preprocessing.sequence.pad_sequences(input_sequence, maxlen=MAX_SEQUENCE_LENGTH)

    result = model.predict(padded_sequence)
    predicted_tag = lbl_encoder.inverse_transform([np.argmax(result)])[0]

    with open(DATA_FILE) as file:
        data = json.load(file)

    response = 'Sorry, I did not understand that.'  

    for intent in data['intents']:
        if intent['intent'] == predicted_tag:
            response = np.random.choice(intent['responses'])
            break

    return jsonify({'message': response})


    chats = [
        {'user': True, 'message': prompt},
        {'user': False, 'message': response}
    ]
    return render_template('index.html', chats=chats)


if __name__ == '__main__':
    app.run()


C:\Users\Abdul Rehman Qureshi\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Jan/2024 12:49:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2024 12:49:01] "GET /style.css HTTP/1.1" 404 -
127.0.0.1 - - [02/Jan/2024 12:49:02] "GET /script.js HTTP/1.1" 404 -


1/1 [==============================] - 0s 175ms/step


127.0.0.1 - - [02/Jan/2024 12:49:55] "POST /chat HTTP/1.1" 200 -


...

127.0.0.1 - - [02/Jan/2024 12:54:31] "POST /chat HTTP/1.1" 200 -


1/1 [==============================] - 0s 40ms/step


127.0.0.1 - - [02/Jan/2024 12:54:43] "POST /chat HTTP/1.1" 200 -


1/1 [==============================] - 0s 33ms/step


127.0.0.1 - - [02/Jan/2024 12:55:10] "POST /chat HTTP/1.1" 200 -
